In [1]:
import time
import nltk
import re
import glob
import os


In [3]:
#####################        DO NOT RUN UNLESS YOU HAVE A FEW HOURS        ################################
#Open pdf files for all years in years list and covert them to a python string


load_path = "./case_txts/"
save_path = "./case_txts/"
years = ["2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012",
        "2013", "2014", "2015", "2016"]

case_str_list = []
counter = 0

for year in years:
    path = load_path
    for filename in os.listdir(path):
        if re.search("raw.txt", filename):
            print "Cleaning...", filename,
#             case_str = "OA" + re.sub(".pdf", "", filename)
#             case_str = re.sub("-", "_", case_str)
#             case_str_list.append(case_str)
            counter +=1
            print filename, counter
            
#             completeName = os.path.join(save_path, case_str_list[-1] +"_raw.txt")
            
#             with open(completeName, "w") as text_file:
#                 text_file.write(case_str)
#                 #print filename, "Raw txt file creation complete"
            
#             #print "Cleaning..."
#             case_str_clean = re.sub("\n\n*", "", case_str)
#             case_str_clean = re.sub("Alderson Reporting Company", "", case_str_clean)
#             case_str_clean = re.sub("Official - Subject to Final Review", "", case_str_clean)
#             case_str_clean = re.sub("\d*", "", case_str_clean)
#             case_str_clean = re.sub("\s\s*", " ", case_str_clean)
#             print filename, "Cleaning Complete!"

#             completeName = os.path.join(save_path, case_str_list[-1] +"_clean.txt")
#             with open(completeName, "w") as text_file:
#                 text_file.write(case_str_clean)
#                 #print filename, "Clean txt file creation complete"
            
#             counter += 1
#             print counter, " cases converted so far"
        
#     print "Year Complete"
        
# Create ZIP file of all case txt files        
# ! cd case_txts; zip -r ../case_txts.zip ../case_txts/*



Cleaning... OA05_1541_raw.txt OA05_1541_raw.txt 1
Cleaning... OA00_1737_raw.txt OA00_1737_raw.txt 2
Cleaning... OA01_400_raw.txt OA01_400_raw.txt 3
Cleaning... OA04_277_raw.txt OA04_277_raw.txt 4
Cleaning... OA09_1036_raw.txt OA09_1036_raw.txt 5
Cleaning... OA03_409_raw.txt OA03_409_raw.txt 6
Cleaning... OA09_11328_raw.txt OA09_11328_raw.txt 7
Cleaning... OA01_1572_raw.txt OA01_1572_raw.txt 8
Cleaning... OA07_588_raw.txt OA07_588_raw.txt 9
Cleaning... OA15_274_d18e_raw.txt OA15_274_d18e_raw.txt 10
Cleaning... OA14_10154_g31h_raw.txt OA14_10154_g31h_raw.txt 11
Cleaning... OA03_388_raw.txt OA03_388_raw.txt 12
Cleaning... OA15_5238_0pm1_raw.txt OA15_5238_0pm1_raw.txt 13
Cleaning... OA10_188_raw.txt OA10_188_raw.txt 14
Cleaning... OA11_393_raw.txt OA11_393_raw.txt 15
Cleaning... OA04_473b_raw.txt OA04_473b_raw.txt 16
Cleaning... OA04_5462_raw.txt OA04_5462_raw.txt 17
Cleaning... OA09_1156_raw.txt OA09_1156_raw.txt 18
Cleaning... OA12_1038_0pm1_raw.txt OA12_1038_0pm1_raw.txt 19
Cleaning... 

In [3]:

import os
import csv
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


# get file names of all text files
file_names = []
for file in os.listdir("./case_txts"):
    if file.endswith("_clean.txt"):
        file_names.append(file)

# open all texts and import into 1 list with each docket being 1 item in the list
texts = []
for f in file_names:
    docket = open("./case_txts/"+f, 'r')
    texts.append(docket.readlines())
    
# get list of verdicts
verdict = []
verdict_csv = csv.reader(open('SCDB_2017_01_caseCentered_Citation.csv'))
for row in verdict_csv:
    verdict.append([row[13], row[36]])
    
# clean up file name to get docket number
##### fix dockets with the word 'orig' in title
docket_number = []
for x in range(0, len(file_names)):
    new_file_name = file_names[x].strip('_clean.txt')
    new_file_name = new_file_name.strip('OA')
    if new_file_name.count('_') >= 2:
        new_file_name = new_file_name[:new_file_name.find('_', 4)]
    new_file_name = new_file_name.replace('_', '-')
    docket_number.append(new_file_name)

# combine verdict (y variable) with the index of the texts (x variable)
# 31 docket numbers that were not clean(includes random characters or can't find it at all) 
# so for the sake of simplicity for now I'm just excluding them from the training set
winning_party = []
texts_cleaned = []
for x in range(0, len(docket_number)):
    for y in range(0, len(verdict)):
        if verdict[y][0] == docket_number[x]:
            winning_party.append([docket_number[x], verdict[y][1]])
            texts_cleaned.append(texts[x])

# flatten texts so it can be used in CountVectorizer
texts_cleaned = [item for sublist in texts_cleaned for item in sublist]
texts_cleaned = np.array(texts_cleaned)

# for reference:
# verdict value 0 = no favorable disposition for petitioning part apparent
# verdict value 1 = petitioning party received a favorable disposition
# verdict value 2 = favorable disposition for petitioning party unclear
# get only the verdict to use for y variable
verdict_cleaned = []
for y in range(0, len(winning_party)):
    verdict_cleaned.append(winning_party[y][1])
verdict_cleaned = np.array(verdict_cleaned)

#separate text and verdicts into train and test sets
train_text = texts_cleaned[len(texts_cleaned)/3:]
test_text = texts_cleaned[:len(texts_cleaned)/3]
train_verdict = verdict_cleaned[len(texts_cleaned)/3:]
test_verdict = verdict_cleaned[:len(verdict_cleaned)/3]

# use CountVectorizer 
# could optimize by changing min_df only take counts above n
count_vect = CountVectorizer(min_df = 5)
x_train_counts = count_vect.fit_transform(train_text)

# tf-idf to normalize word frequency
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
x_train_tfidf.toarray()

# transform test data
x_test_counts = count_vect.transform(test_text)
x_test_tfidf = tfidf_transformer.transform(x_test_counts)
x_test_tfidf.toarray()

# Multinomial Naive Baynes
# could optimize by changing alpha
clf = MultinomialNB(alpha = 0.1).fit(x_train_tfidf, train_verdict)
naive_baynes_prediction = clf.predict(x_test_tfidf)
nb_accuracy = np.mean(naive_baynes_prediction == test_verdict)
print 'Accuracy of Bag-of-Words Multinomial Naive Baynes Model:', round(nb_accuracy,4)

# Random Forest
# could optimize by changing n_estimators
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(x_train_tfidf, train_verdict)
random_forest_prediction = forest.predict(x_test_tfidf)
rf_accuracy = np.mean(random_forest_prediction == test_verdict)
print 'Accuracy of Bag-of-Words Random Forest Model:', round(rf_accuracy,4)

# logistic regression
# could optimize by changing C
lr_model = LogisticRegression(C = 1, penalty = "l2")
lr_model_train = lr_model.fit(x_train_tfidf, train_verdict)
lr_prediction = lr_model.predict(x_test_tfidf)
lr_accuracy = np.mean(lr_prediction == test_verdict)
print 'Accuracy of Bag-of-Words Logistic Regression Model:', round(lr_accuracy,4)

NameError: name 'case_str_list' is not defined

In [127]:
#print case_str_clean

In [128]:
tokens_clean = nltk.sent_tokenize(case_str_clean.decode('utf-8'))
for i in range(len(tokens_clean)):
    print tokens_clean[i]
    print "_____________________"

  IN THE SUPREME COURT OF THE UNITED STATES - - - - - - - - - - - - - - - -X ELAINE L. CHAO, SECRETARY OF : LABOR, : Petitioner : v. : No.
_____________________
- MALLARD BAY DRILLING, INC. : - - - - - - - - - - - - - - - -X Courtroom Constitution Avenue, N.W.
_____________________
Washington, D.C. Wednesday, October , The above-entitled matter came on for oral argument before the Supreme Court of the United States at : a.m.
_____________________
APPEARANCES: MATTHEW D. ROBERTS, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the Petitioner.
_____________________
PATRICK J. VETERS, ESQ., New Orleans, Louisiana; on behalf of the Respondent.
_____________________
th Street, N.W.
_____________________
Suite --FOR-DEPO Washington, DC C O N T E N T S ORAL ARGUMENT OF PAGE MATTHEW D. ROBERTS, ESQ.
_____________________
On behalf of the Petitioner PATRICK J. VETERS, ESQ.
_____________________
On behalf of the Respondent REBUTTAL ARGUMENT OF MATT

In [104]:
print tokens_clean

[u'Official\xa0\xad\xa0Subject\xa0to\xa0Final\xa0Review \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0IN THE SUPREME COURT OF THE UNITED STATES \xa0\xa0\xa0\xa0\xa0\xa0\xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad x \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0RJR NABISCO, INC., ET AL., \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0: \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Petitioners \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0: No.', u'\xad \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0v. \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0THE EUROPEAN COMMUNITY, ET AL., \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Respondents.', u': : : \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad \xad x \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\